## <center> Packet Sniffing and Spoofing </center>

### How Packets Are Received

#### Network Interface Card
- Network Interface Cards (NIC) provide the link (physival or virtual) between machines and the network. 
- Each NIC has a hardware address called MAC (media access control) address.
- Ethernet and Wifi are broadcast medium by nature
  - All machines are connected to a single shared medium
  - Every NIC on the network will *hear* everything on the wire

<center> <img src="figure/packet_sniffing/nic-to-application.png" width="600"/>

- 1: A frame is copied into NIC's memory.
- 2: If the address matches the NIC's MAC address, the frame is further copied into the *Ring Buffer* via Direct Memory Access (DMA).
- 3: NIC interrupts CPU to inform about the availability of a new packet.
- 4: The CPU copies all the packets from the buffer into a queue. 
- 5: Based on the protocol, the kernel invokes different handler functions to process the data and send the packts to the applications. 

<center> <img src="figure/packet_sniffing/kernel_networking.jpg" width="600"/>

#### Promiscuous Mode
- Frames (packets) not addressed to a NIC are discarded instead of being passed to the CPU for processing. 
- Most NIC has a special mode called **promiscuous mode**: NIC passes every frame received from the network to the kernel regardless of the destination MAC address. 
- **Promiscuous mode** requires root privilege. 

#### Monitor Mode
- Similar to Promiscuous Mode, but is for wireless network

#### [BSD Packet Filter](https://www.tcpdump.org/papers/bpf-usenix93.pdf)
- Developed by Steven McCanne and Van Jacobson at Lawrence Berkeley Laboratory
- Presented at USENIX '93
- Provided a filtering mechanism for packets' types at socket level (inside kernel space)


### Packet Sniffing

#### Receiving Packets using Sockets

In [ ]:
#include <stdio.h>
#include <string.h>
#include <sys/socket.h>
#include <netinet/ip.h>

void main()
{
    struct sockaddr_in server;
    struct sockaddr_in client;
    int clientlen;
    char buf[1500];

    /* create a new socket */

    int sock = socket(AF_INET, SOCK_DGRAM, IPPROTO_UDP);

    /* create a data structure containing information about the server to be bound to the socket */
    memset((char *) &server, 0, sizeof(server));
    server.sin_family = AF_INET;
    server.sin_addr.s_addr = htonl(INADDR_ANY);
    server.sin_port = htons(9090);

    if (bind(sock, (struct sockaddr *) &server, sizeof(server)) < 0)
        error("ERROR on binding");

    /* the server receives and processes UDP packets through the socket */
    while (1) {
        bzero(buf, 1500);
        
        recvfrom(sock, buf, 1500-1, 0, 
               (struct sockaddr *) &client, &clientlen);
        printf("%s\n", buf);
    }
    close(sock);
}


    - AF_INET: 
      - http://man7.org/linux/man-pages/man2/socket.2.html       
      - https://www.ibm.com/support/knowledgecenter/en/ssw_ibm_i_71/rzab6/cafinet.htm
    - SOCK_DGRAM: Socket datagrame (for UDP)
    - socket: https://linux.die.net/man/7/ip
    - recvfrom: https://linux.die.net/man/2/recvfrom
    - bind: http://man7.org/linux/man-pages/man2/bind.2.html
    - bzero: http://man7.org/linux/man-pages/man3/bzero.3.html
    

<center> <img src="figure/packet_sniffing/kernel_networking_C.jpg" width="600"/>

- Open a terminal 
- SSH into a lab instance and run the followings:

```
$ cp -R /local/repository/setup_scripts/packet_sniffing/codes .
$ gcc ~/codes/receive_packet.c
$ ./a.out
```

#### Sniffing Packets using Raw Sockets

In [ ]:
#include <sys/socket.h>
#include <linux/if_packet.h>
#include <net/ethernet.h>
#include <stdio.h>

int main() {
    int PACKET_LEN = 512;
    char buffer[PACKET_LEN];
    struct sockaddr saddr;
    struct packet_mreq mr;

    // Create the raw socket
    int sock = socket(AF_PACKET, SOCK_RAW, htons(ETH_P_ALL));  

    // Turn on the promiscuous mode. 
    mr.mr_type = PACKET_MR_PROMISC;                           
    setsockopt(sock, SOL_PACKET, PACKET_ADD_MEMBERSHIP, &mr,  
                     sizeof(mr));

    // Getting captured packets
    while (1) {
        int data_size=recvfrom(sock, buffer, PACKET_LEN, 0,  
                 &saddr, (socklen_t*)sizeof(saddr));
        if(data_size) printf("Got one packet\n");
    }

    close(sock);
    return 0;
}

- Open a terminal 
- SSH into a lab instance and run the followings:

```
$ gcc ~/codes/sniff_packet_raw_socket.c
$ ./a.out
```

#### Sniffing Packets using the *pcap* API

In [ ]:
#include <pcap.h>
#include <stdio.h>

void got_packet(u_char *args, const struct pcap_pkthdr *header, 
        const u_char *packet)
{
   printf("Got a packet\n");
}

int main()
{
  pcap_t *handle;
  char errbuf[PCAP_ERRBUF_SIZE];
  struct bpf_program fp;
  char filter_exp[] = "ip proto icmp";
  bpf_u_int32 net;

  // Step 1: Open live pcap session on NIC with name eth3
  handle = pcap_open_live("eth1", BUFSIZ, 1, 1000, errbuf); 

  // Step 2: Compile filter_exp into BPF psuedo-code
  pcap_compile(handle, &fp, filter_exp, 0, net);      
  pcap_setfilter(handle, &fp);                             

  // Step 3: Capture packets
  pcap_loop(handle, -1, got_packet, NULL);                

  pcap_close(handle);   //Close the handle 
  return 0;
}


```

$ gcc -lpcap -o sniffer ~/codes/sniff_packet_pcap.c
$ ./sniffer
$ sudo ./sniffer

```

#### Exercise

- Modify `sniff_packet_pcap`'s filter to be `src port 9090`, recompile, and rerun.
- Count how many packets received in a batch? 
- Compare with instructor's message generation script. 


#### Processing Captured Packet

- `void got_packet(u_char *args, const struct pcap_pkthdr *header, const u_char *packet)`
- **const u_char  \*packet**: buffer holding the packet's content.
- We need to find the correct offsets of different field. 
- This can be don eusing C's `struct` and type casting

<center> <img src="figure/packet_sniffing/udp_header.gif" width="600"/>

In [ ]:
```c

#include <pcap.h>
#include <stdio.h>
#include <arpa/inet.h>
#include <net/ethernet.h>

/* Ethernet header */
struct ethheader {
  u_char  ether_dhost[ETHER_ADDR_LEN]; /* destination host address */
  u_char  ether_shost[ETHER_ADDR_LEN]; /* source host address */
  u_short ether_type;                  /* IP? ARP? RARP? etc */
};

/* IP Header */
struct ipheader {
  unsigned char      iph_ihl:4, //IP header length
                     iph_ver:4; //IP version
  unsigned char      iph_tos; //Type of service
  unsigned short int iph_len; //IP Packet length (data + header)
  unsigned short int iph_ident; //Identification
  unsigned short int iph_flag:3, //Fragmentation flags
                     iph_offset:13; //Flags offset
  unsigned char      iph_ttl; //Time to Live
  unsigned char      iph_protocol; //Protocol type
  unsigned short int iph_chksum; //IP datagram checksum
  struct  in_addr    iph_sourceip; //Source IP address 
  struct  in_addr    iph_destip;   //Destination IP address 
};

void got_packet(u_char *args, const struct pcap_pkthdr *header, 
                              const u_char *packet)
{
  struct ethheader *eth = (struct ethheader *)packet;

  if (ntohs(eth->ether_type) == 0x0800) { // 0x0800 is IP type
    struct ipheader * ip = (struct ipheader *)
                           (packet + sizeof(struct ethheader)); 

    printf("       From: %s\n", inet_ntoa(ip->iph_sourceip));  
    printf("         To: %s\n", inet_ntoa(ip->iph_destip));   

    /* determine protocol */
    switch(ip->iph_protocol) {                               
        case IPPROTO_TCP:
            printf("   Protocol: TCP\n");
            return;
        case IPPROTO_UDP:
            printf("   Protocol: UDP\n");
            return;
        case IPPROTO_ICMP:
            printf("   Protocol: ICMP\n");
            return;
        default:
            printf("   Protocol: others\n");
            return;
    }
  }
}

int main()
{
  pcap_t *handle;
  char errbuf[PCAP_ERRBUF_SIZE];
  struct bpf_program fp;
  char filter_exp[] = "ip proto icmp";
  bpf_u_int32 net;

  // Step 1: Open live pcap session on NIC with name eth3
  handle = pcap_open_live("eth1", BUFSIZ, 1, 1000, errbuf); 

  // Step 2: Compile filter_exp into BPF psuedo-code
  pcap_compile(handle, &fp, filter_exp, 0, net);      
  pcap_setfilter(handle, &fp);                             

  // Step 3: Capture packets
  pcap_loop(handle, -1, got_packet, NULL);                

  pcap_close(handle);   //Close the handle 
  return 0;
}

```

#### Excercise

Given that a UDP Datagram has the following structure:

- 2 bytes for source port number
- 2 bytes for destination port number
- 2 bytes for length of data
- 2 bytes for UDP checksum

Modify the above source code so that the content of the UDP message can be displayed. 

```c

/* UDP Header */
struct udpheader
{
  u_int16_t udp_sport;           /* source port */
  u_int16_t udp_dport;           /* destination port */
  u_int16_t udp_ulen;            /* udp length */
  u_int16_t udp_sum;             /* udp checksum */
};

```


and

```c

struct udpheader *udp = (struct udpheader *)
                        (packet + sizeof(struct ethheader) + sizeof(struct ipheader));
char *msg = malloc(udp->udp_ulen * sizeof(char));
msg = packet +  sizeof(struct ethheader) + sizeof(struct ipheader) + sizeof(struct udpheader);
printf(" Message: %s\n", msg);

```

#### Send Normal Packets using Socket

```c

#include <stdio.h>
#include <string.h>
#include <sys/socket.h>
#include <netinet/ip.h>

void main()
{
    struct sockaddr_in dest_info;
    char *data = "UDP message\n";

    // Step 1: Create a network socket
    int sock = socket(AF_INET, SOCK_DGRAM, IPPROTO_UDP);

    // Step 2: Provide information about destination.
    memset((char *) &dest_info, 0, sizeof(dest_info));
    dest_info.sin_family = AF_INET;
    dest_info.sin_addr.s_addr = inet_addr("192.168.1.1");
    dest_info.sin_port = htons(9090);

    // Step 3: Send out the packet.
    sendto(sock, data, strlen(data), 0,
                 (struct sockaddr *)&dest_info, sizeof(dest_info));
    close(sock);
}


```

- Instructor opens a listening post at 9090: `nc -luv 9090`
- Customize the data message in `send_packet_socket.c` to include your name. 
- Compile and run `send_packet_socket.c`.

#### Sending Spoofed Packets using Raw Socket (ICMP)

```c

#include <stdio.h>
#include <string.h>
#include <sys/socket.h>
#include <netinet/ip.h>

/* ICMP Header  */
struct icmpheader {
  unsigned char icmp_type; // ICMP message type
  unsigned char icmp_code; // Error code
  unsigned short int icmp_chksum; //Checksum for ICMP Header and data
  unsigned short int icmp_id;     //Used for identifying request
  unsigned short int icmp_seq;    //Sequence number
};

/* IP Header */
struct ipheader {
  unsigned char      iph_ihl:4, //IP header length
                     iph_ver:4; //IP version
  unsigned char      iph_tos; //Type of service
  unsigned short int iph_len; //IP Packet length (data + header)
  unsigned short int iph_ident; //Identification
  unsigned short int iph_flag:3, //Fragmentation flags
                     iph_offset:13; //Flags offset
  unsigned char      iph_ttl; //Time to Live
  unsigned char      iph_protocol; //Protocol type
  unsigned short int iph_chksum; //IP datagram checksum
  struct  in_addr    iph_sourceip; //Source IP address 
  struct  in_addr    iph_destip;   //Destination IP address 
};

/*************************************************************
  Given an IP packet, send it out using a raw socket. 
**************************************************************/
void send_raw_ip_packet(struct ipheader* ip)
{
    struct sockaddr_in dest_info;
    int enable = 1;

    // Step 1: Create a raw network socket.
    int sock = socket(AF_INET, SOCK_RAW, IPPROTO_RAW);

    // Step 2: Set socket option.
    setsockopt(sock, IPPROTO_IP, IP_HDRINCL, 
                     &enable, sizeof(enable));

    // Step 3: Provide needed information about destination.
    dest_info.sin_family = AF_INET;
    dest_info.sin_addr = ip->iph_destip;

    // Step 4: Send the packet out.
    sendto(sock, ip, ntohs(ip->iph_len), 0, 
           (struct sockaddr *)&dest_info, sizeof(dest_info));
    close(sock);
}

unsigned short in_cksum (unsigned short *buf, int length)
{
   unsigned short *w = buf;
   int nleft = length;
   int sum = 0;
   unsigned short temp=0;

   /*
    * The algorithm uses a 32 bit accumulator (sum), adds
    * sequential 16 bit words to it, and at the end, folds back all 
    * the carry bits from the top 16 bits into the lower 16 bits.
    */
   while (nleft > 1)  {
       sum += *w++;
       nleft -= 2;
   }

   /* treat the odd byte at the end, if any */
   if (nleft == 1) {
        *(u_char *)(&temp) = *(u_char *)w ;
        sum += temp;
   }

   /* add back carry outs from top 16 bits to low 16 bits */
   sum = (sum >> 16) + (sum & 0xffff);  // add hi 16 to low 16 
   sum += (sum >> 16);                  // add carry 
   return (unsigned short)(~sum);
}

int main() {
   char buffer[1500];

   memset(buffer, 0, 1500);

   /*********************************************************
      Step 1: Fill in the ICMP header.
    ********************************************************/
   struct icmpheader *icmp = (struct icmpheader *) 
                             (buffer + sizeof(struct ipheader));
   icmp->icmp_type = 8; //ICMP Type: 8 is request, 0 is reply.

   // Calculate the checksum for integrity
   icmp->icmp_chksum = 0;
   icmp->icmp_chksum = in_cksum((unsigned short *)icmp, 
                                 sizeof(struct icmpheader));

   /*********************************************************
      Step 2: Fill in the IP header.
    ********************************************************/
   struct ipheader *ip = (struct ipheader *) buffer;
   ip->iph_ver = 4;
   ip->iph_ihl = 5;
   ip->iph_ttl = 20;
   ip->iph_sourceip.s_addr = inet_addr("1.2.3.4");
   ip->iph_destip.s_addr = inet_addr("192.168.1.1");
   ip->iph_protocol = IPPROTO_ICMP; 
   ip->iph_len = htons(sizeof(struct ipheader) + 
                       sizeof(struct icmpheader));

   /*********************************************************
      Step 3: Finally, send the spoofed packet
    ********************************************************/
   send_raw_ip_packet (ip);

   return 0;
}

```

While instructor runs `sudo tcpdump -i eth1 'icmp' -v`, perform the followings:

```
$ gcc ~/codes/send_packet_raw_icmp.c
$ ./a.out
$ sudo ./a.out
```